1．实现公开的MNIST手写数字数据集的分类或自选数据的分类；包括数据获取、训练集和测试集创建、工作空间创建、训练数据导入； 

2．调用scikit-learn的基本库，自己实现利用Parzen窗、Gaussian 分布和KNN实现MINIST的概率密度估计并进行分类,及基于训练好模型的测试、实验报告撰写。

3．利用第三方优化工具包实现参数的优化，并与scikit-learn中标准算法进行自己实现算法的时间复杂度的对比。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn import manifold
from sklearn.metrics import accuracy_score

# 获取数据
data = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
pixel_values, targets = data
targets = targets.astype(int)
pixel_array = pixel_values.to_numpy()

/opt/homebrew/Caskroom/miniconda/base/envs/datasci/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
# 使用 t-SNE 进行降维
tsne = manifold.TSNE(n_components=2, random_state=42, perplexity=25)
transformed_data = tsne.fit_transform(pixel_array)

In [8]:
# 创建 DataFrame
tsne_df = pd.DataFrame(np.column_stack((transformed_data, targets)),
                       columns=["x", "y", "targets"])
tsne_df.loc[:, "targets"] = tsne_df.targets.astype(int)

# 划分训练集和测试集
X_train = transformed_data[:60000]
y_train = targets[:60000]
X_test = transformed_data[60000:]
y_test = targets[60000:]

In [27]:
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist

# 执行 Parzen 窗口密度估计的函数

def parzen_window_estimation_optimized(X_train, X_test, h):
    # 计算所有测试样本和所有训练样本之间的距离
    distances = cdist(X_test, X_train, 'euclidean')
    # 计算 Parzen 窗口
    kernel_values = np.exp(-0.5 * (distances / h) ** 2) / (np.sqrt(2 * np.pi) * h)
    # 求每个测试样本的平均密度
    return kernel_values.mean(axis=1)

# 根据 Parzen 窗口估计进行分类的函数
def classify_parzen_window(X_train, y_train, X_test, h):
    unique_classes = np.unique(y_train)
    predictions = np.zeros(X_test.shape[0])
    for class_val in unique_classes:
        # 对每个类应用 Parzen 窗口估计
        class_mask = y_train == class_val
        pdf_estimates = parzen_window_estimation_optimized(X_train[class_mask], X_test, h)
        # 假设类别概率相等，取似然度最大值
        if class_val == 0 or pdf_estimates > predictions:
            predictions = pdf_estimates
            predicted_class = class_val
    return predicted_class

In [48]:
# 为 Parzen 窗口设置带宽参数
h = 0.42813323987193935 # 这是在一次失败的实验中得到的最佳参数，其实0.5左右的参数效果差不多。 

predictions = []
n_test_samples = len(X_test)
print_interval = 1000  # 每处理1000张图片打印一次进度
print(n_test_samples)

10000


In [49]:
for i in range(n_test_samples):
    prediction = classify_parzen_window(X_train, y_train, X_test[i].reshape(1, -1), h)
    predictions.append(prediction)

    # 每处理1000张图片时打印进度
    if (i + 1) % print_interval == 0 or i == n_test_samples - 1:
        print(f"已处理 {i + 1} / {n_test_samples} 张图片")

predictions = np.array(predictions)

已处理 1000 / 10000 张图片
已处理 2000 / 10000 张图片
已处理 3000 / 10000 张图片
已处理 4000 / 10000 张图片
已处理 5000 / 10000 张图片
已处理 6000 / 10000 张图片
已处理 7000 / 10000 张图片
已处理 8000 / 10000 张图片
已处理 9000 / 10000 张图片
已处理 10000 / 10000 张图片


In [50]:
# 计算准确率
accuracy = accuracy_score(y_test, predictions)
print(f'准确率: {accuracy}')

准确率: 0.9731


In [58]:
from sklearn.neighbors import KNeighborsClassifier

# 创建 k-NN 分类器实例
knn = KNeighborsClassifier(n_neighbors=3)

# 训练分类器
knn.fit(X_train, y_train)

# 进行预测
predictions = knn.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, predictions)
print(f'准确率: {accuracy:.5f}')

准确率: 0.97260
